#***First Section

##1)Using python functions

###Importing Libraries

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import models, layers, activations
from sklearn.model_selection import train_test_split

print(tf.__version__)
# set random seed to get reproducible results
np.random.seed(0)
tf.random.set_seed(1)

# only for Google Colab compatibiity
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

2.4.1


###Creating Dataset

In [2]:
max_bits = 8
n_samples = 100000

# samples in decimal form
samples = np.random.randint(np.power(2, max_bits-1), size=(n_samples, 2))
summed_samples = np.sum(samples, axis=1)

# convert samples to binary representation
samples_binary_repr = [[np.binary_repr(a, width=max_bits), np.binary_repr(b, width=max_bits)] for a,b in samples]
summed_binary_repr = [np.binary_repr(c, width=max_bits) for c in summed_samples]

x_str = np.array([[list(a), list(b)] for a, b in samples_binary_repr])
y_str = np.array([list(c) for c in summed_binary_repr])

# flip binary representation to get increasing significant bit
x_flipped = np.flip(x_str, axis=-1)
y_flipped = np.flip(y_str, axis=-1)

# convert string to numbers
x = np.transpose((x_flipped == '1')*1, axes=(0, 2, 1))
y = (y_flipped == '1')*1

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)

###Structure of RNN

In [3]:
class FullAdderCell(layers.Layer):
    def __init__(self, hidden_units, **kwargs):
        super(FullAdderCell, self).__init__(**kwargs)
        self.units = 1
        self.state_size = 1
        self.hidden_units = hidden_units

    def build(self, input_shape):
        self.hidden_kernel = self.add_weight(shape=(input_shape[-1] + self.state_size, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_kernel')
        self.hidden_bias = self.add_weight(shape=(1, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_bias')
        self.output_kernel = self.add_weight(shape=(self.hidden_units, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_kernel')
        self.output_bias = self.add_weight(shape=(1, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_bias')
        self.built = True

    def call(self, inputs, states):
        x = tf.concat([inputs, states[0]], axis=-1)
        h = tf.keras.activations.tanh(tf.matmul(x, self.hidden_kernel) + self.hidden_bias)
        o_s = tf.keras.activations.sigmoid(tf.matmul(h, self.output_kernel) + self.output_bias)
        output = o_s[:, :self.units]
        state = o_s[:, self.units:]
        return output, [state]


###Creating Model

In [4]:
model = tf.keras.Sequential(name='full_adder')
model.add(layers.RNN(FullAdderCell(3), return_sequences=True, input_shape=(None, 2)))

model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=5)
scores = model.evaluate(x_test, y_test, verbose=2)

Model: "full_adder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn (RNN)                    (None, None, 1)           20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
2813/2813 [==============================] - 9s 3ms/step - loss: 0.6931 - accuracy: 0.5012
Epoch 2/5
2813/2813 [==============================] - 8s 3ms/step - loss: 0.4994 - accuracy: 0.7993
Epoch 3/5
2813/2813 [==============================] - 8s 3ms/step - loss: 0.0921 - accuracy: 1.0000
Epoch 4/5
2813/2813 [==============================] - 7s 3ms/step - loss: 0.0184 - accuracy: 1.0000
Epoch 5/5
2813/2813 [==============================] - 7s 3ms/step - loss: 0.0046 - accuracy: 1.0000
313/313 - 1s - loss: 0.0017 - accuracy: 1.0000


###Testing the model

In [5]:
max_bits = 8

a = np.random.randint(np.power(2, max_bits-1))
b = np.random.randint(np.power(2, max_bits-1))

a_bin = np.float32(1) * (np.flip(list(np.binary_repr(a, width=max_bits)), axis=-1) == '1')
b_bin = np.float32(1) * (np.flip(list(np.binary_repr(b, width=max_bits)), axis=-1) == '1')

print('a: {}, b: {}'.format(a, b))
print('binary representations -> a: {}, b: {}'.format(a_bin, b_bin))

a_b = np.stack((a_bin, b_bin), axis=-1).reshape(1,-1,2)
print('a_b: {}'.format(a_b))

a: 48, b: 10
binary representations -> a: [0. 0. 0. 0. 1. 1. 0. 0.], b: [0. 1. 0. 1. 0. 0. 0. 0.]
a_b: [[[0. 0.]
  [0. 1.]
  [0. 0.]
  [0. 1.]
  [1. 0.]
  [1. 0.]
  [0. 0.]
  [0. 0.]]]


In [6]:
predictions = model(a_b).numpy().flatten()

summed_bin = 1 * (predictions > 0.5)
summed = np.packbits(np.flip(summed_bin , axis=-1))[0]
print('predictions: {}'.format(predictions))
print('binary representations -> summed: {}'.format(summed_bin))
print('summed: {}'.format(summed))

predictions: [0.00121176 0.9986558  0.00121257 0.9986558  0.9986553  0.9986553
 0.00121257 0.00121173]
binary representations -> summed: [0 1 0 1 1 1 0 0]
summed: 58


##2)Building from scratch

### Importing Libraries

In [7]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import itertools

sns.set_style('darkgrid')
np.random.seed(seed=1)

### Create Dataset

In [8]:
# Number of training samples
nb_train = 2000  
# Length of the binary sequence
sequence_len = 7  

In [9]:
def create_dataset(nb_samples, sequence_len):

    # Maximum integer that can be added
    max_int = 2**(sequence_len-1) 
    
    # Transform integer in binary format
    format_str = '{:0' + str(sequence_len) + 'b}'

    # Add 2 binary numbers
    nb_inputs = 2  

    # Result is 1 binary number
    nb_outputs = 1  

    # Input samples
    X = np.zeros((nb_samples, sequence_len, nb_inputs))

    # Target samples
    T = np.zeros((nb_samples, sequence_len, nb_outputs))

    # Fill up the input and target matrix
    for i in range(nb_samples):
        
        # Generate random numbers to add
        nb1 = np.random.randint(0, max_int)
        nb2 = np.random.randint(0, max_int)

        # Fill current input and target row.
        X[i,:,0] = list(
            reversed([int(b) for b in format_str.format(nb1)]))
        X[i,:,1] = list(
            reversed([int(b) for b in format_str.format(nb2)]))
        T[i,:,0] = list(
            reversed([int(b) for b in format_str.format(nb1+nb2)]))
    return X, T

In [10]:
# Create training samples
X_train, T_train = create_dataset(nb_train, sequence_len)

print(f'X_train tensor shape: {X_train.shape}')
print(f'T_train tensor shape: {T_train.shape}')

X_train tensor shape: (2000, 7, 2)
T_train tensor shape: (2000, 7, 1)


### Sample of input and output

In [11]:
def printSample(x1, x2, t, y=None):

    x1 = ''.join([str(int(d)) for d in x1])
    x1_r = int(''.join(reversed(x1)), 2)

    x2 = ''.join([str(int(d)) for d in x2])
    x2_r = int(''.join(reversed(x2)), 2)

    t = ''.join([str(int(d[0])) for d in t])
    t_r = int(''.join(reversed(t)), 2)

    if not y is None:
        y = ''.join([str(int(d[0])) for d in y])

    print(f'x1:   {x1:s}   {x1_r:2d}')
    print(f'x2: + {x2:s}   {x2_r:2d}')
    print(f'      -------   --')
    print(f't:  = {t:s}   {t_r:2d}')
    if not y is None:
        print(f'y:  = {y:s}')

In [12]:
printSample(X_train[0,:,0], X_train[0,:,1], T_train[0,:,:])

x1:   1010010   37
x2: + 1101010   43
      -------   --
t:  = 0000101   80


### Linear Transformation

In [13]:
class TensorLinear(object):
    def __init__(self, n_in, n_out, tensor_order, W=None, b=None):
        a = np.sqrt(6.0 / (n_in + n_out))
        self.W = (np.random.uniform(-a, a, (n_in, n_out)) 
                  if W is None else W)
        self.b = (np.zeros((n_out)) if b is None else b)
        self.bpAxes = tuple(range(tensor_order-1))

    def forward(self, X):
        return np.tensordot(X, self.W, axes=((-1),(0))) + self.b

    def backward(self, X, gY):
        gW = np.tensordot(X, gY, axes=(self.bpAxes, self.bpAxes))
        gB = np.sum(gY, axis=self.bpAxes)
        gX = np.tensordot(gY, self.W.T, axes=((-1),(0)))  
        return gX, gW, gB

### Logistic Classification

In [14]:
class LogisticClassifier(object):
   
    def forward(self, X):
        return 1. / (1. + np.exp(-X))
    
    def backward(self, Y, T):
        return (Y - T) / (Y.shape[0] * Y.shape[1])
    
    def loss(self, Y, T):
        return -np.mean((T * np.log(Y)) + ((1-T) * np.log(1-Y)))

### Unfolding the recurrent states


#### Define tanh layer

In [15]:
class TanH(object):
    def forward(self, X):
        return np.tanh(X) 
    
    def backward(self, Y, output_grad):
        gTanh = 1.0 - (Y**2)
        return (gTanh * output_grad)

#### Define internal state update layer

In [16]:
class RecurrentStateUpdate(object):
    def __init__(self, nbStates, W, b):
        self.linear = TensorLinear(nbStates, nbStates, 2, W, b)
        self.tanh = TanH()

    def forward(self, Xk, Sk):
        return self.tanh.forward(Xk + self.linear.forward(Sk))
    
    def backward(self, Sk0, Sk1, output_grad):
        gZ = self.tanh.backward(Sk1, output_grad)
        gSk0, gW, gB = self.linear.backward(Sk0, gZ)
        return gZ, gSk0, gW, gB

#### Define layer that unfolds the states over time

In [17]:
class RecurrentStateUnfold(object):
    def __init__(self, nbStates, nbTimesteps):
        a = np.sqrt(6. / (nbStates * 2))
        self.W = np.random.uniform(-a, a, (nbStates, nbStates))
        self.b = np.zeros((self.W.shape[0])) 
        self.S0 = np.zeros(nbStates) 
        self.nbTimesteps = nbTimesteps 
        self.stateUpdate = RecurrentStateUpdate(
            nbStates, self.W, self.b) 
        
    def forward(self, X):
        S = np.zeros((X.shape[0], X.shape[1]+1, self.W.shape[0]))
        S[:,0,:] = self.S0 
        for k in range(self.nbTimesteps):
            S[:,k+1,:] = self.stateUpdate.forward(X[:,k,:], S[:,k,:])
        return S
    
    def backward(self, X, S, gY):
        gSk = np.zeros_like(gY[:,self.nbTimesteps-1,:])
        gZ = np.zeros_like(X)
        gWSum = np.zeros_like(self.W) 
        gBSum = np.zeros_like(self.b) 
        for k in range(self.nbTimesteps-1, -1, -1):
            gSk += gY[:,k,:]
            gZ[:,k,:], gSk, gW, gB = self.stateUpdate.backward(
                S[:,k,:], S[:,k+1,:], gSk)
            gWSum += gW  
            gBSum += gB  
        gS0 = np.sum(gSk, axis=0)
        return gZ, gWSum, gBSum, gS0

### Network

In [18]:
# Define the full network
class RnnBinaryAdder(object):
    def __init__(self, nb_of_inputs, nb_of_outputs, nb_of_states, 
                 sequence_len):
        self.tensorInput = TensorLinear(nb_of_inputs, nb_of_states, 3)
        self.rnnUnfold = RecurrentStateUnfold(nb_of_states, sequence_len)
        self.tensorOutput = TensorLinear(nb_of_states, nb_of_outputs, 3)
        self.classifier = LogisticClassifier() 
        
    def forward(self, X):
        recIn = self.tensorInput.forward(X)
        S = self.rnnUnfold.forward(recIn)
        Z = self.tensorOutput.forward(S[:,1:sequence_len+1,:])
        Y = self.classifier.forward(Z) 
        return recIn, S, Z, Y
    
    def backward(self, X, Y, recIn, S, T):
        gZ = self.classifier.backward(Y, T)  
        gRecOut, gWout, gBout = self.tensorOutput.backward(
            S[:,1:sequence_len+1,:], gZ)
        gRnnIn, gWrec, gBrec, gS0 = self.rnnUnfold.backward(
            recIn, S, gRecOut)
        gX, gWin, gBin = self.tensorInput.backward(X, gRnnIn)
        return gWout, gBout, gWrec, gBrec, gWin, gBin, gS0
    
    def getOutput(self, X):
        recIn, S, Z, Y = self.forward(X)
        return Y
    
    def getBinaryOutput(self, X):
        return np.around(self.getOutput(X))
    
    def getParamGrads(self, X, T):
        recIn, S, Z, Y = self.forward(X)
        gWout, gBout, gWrec, gBrec, gWin, gBin, gS0 = self.backward(
            X, Y, recIn, S, T)
        return [g for g in itertools.chain(
                np.nditer(gS0),
                np.nditer(gWin),
                np.nditer(gBin),
                np.nditer(gWrec),
                np.nditer(gBrec),
                np.nditer(gWout),
                np.nditer(gBout))]
    
    def loss(self, Y, T):
        return self.classifier.loss(Y, T)
    
    def get_params_iter(self):
        return itertools.chain(
            np.nditer(self.rnnUnfold.S0, op_flags=['readwrite']),
            np.nditer(self.tensorInput.W, op_flags=['readwrite']),
            np.nditer(self.tensorInput.b, op_flags=['readwrite']),
            np.nditer(self.rnnUnfold.W, op_flags=['readwrite']),
            np.nditer(self.rnnUnfold.b, op_flags=['readwrite']),
            np.nditer(self.tensorOutput.W, op_flags=['readwrite']), 
            np.nditer(self.tensorOutput.b, op_flags=['readwrite']))

### Gradient Checking

In [19]:
RNN = RnnBinaryAdder(2, 1, 3, sequence_len)
backprop_grads = RNN.getParamGrads(
    X_train[0:100,:,:], T_train[0:100,:,:])

eps = 1e-7 
for p_idx, param in enumerate(RNN.get_params_iter()):
    grad_backprop = backprop_grads[p_idx]

    param += eps
    plus_loss = RNN.loss(
        RNN.getOutput(X_train[0:100,:,:]), T_train[0:100,:,:])

    param -= 2 * eps
    min_loss = RNN.loss(
        RNN.getOutput(X_train[0:100,:,:]), T_train[0:100,:,:])

    param += eps

    grad_num = (plus_loss - min_loss) / (2*eps)

    if not np.isclose(grad_num, grad_backprop):
        raise ValueError((
            f'Numerical gradient of {grad_num:.6f} is not close '
            f'to the backpropagation gradient of {grad_backprop:.6f}!'
        ))
print('No gradient errors found')

No gradient errors found


In [20]:
lmbd = 0.5  # Rmsprop lambda
learning_rate = 0.05  # Learning rate
momentum_term = 0.80  # Momentum term
eps = 1e-6  # Numerical stability term to prevent division by zero
mb_size = 100  # Size of the minibatches (number of samples)


nb_of_states = 3  # Number of states in the recurrent layer
RNN = RnnBinaryAdder(2, 1, nb_of_states, sequence_len)
nbParameters =  sum(1 for _ in RNN.get_params_iter())
maSquare = [0.0 for _ in range(nbParameters)]
Vs = [0.0 for _ in range(nbParameters)]  
ls_of_loss = [
    RNN.loss(RNN.getOutput(X_train[0:100,:,:]), T_train[0:100,:,:])]

for i in range(5):
    for mb in range(nb_train // mb_size):
        X_mb = X_train[mb:mb+mb_size,:,:] 
        T_mb = T_train[mb:mb+mb_size,:,:]  
        V_tmp = [v * momentum_term for v in Vs]
        for pIdx, P in enumerate(RNN.get_params_iter()):
            P += V_tmp[pIdx]
        backprop_grads = RNN.getParamGrads(X_mb, T_mb)    
        for pIdx, P in enumerate(RNN.get_params_iter()):
            maSquare[pIdx] = lmbd * maSquare[pIdx] + (
                1-lmbd) * backprop_grads[pIdx]**2
            pGradNorm = ((
                learning_rate * backprop_grads[pIdx]) / np.sqrt(
                maSquare[pIdx]) + eps)
            Vs[pIdx] = V_tmp[pIdx] - pGradNorm     
            P -= pGradNorm  
        ls_of_loss.append(RNN.loss(RNN.getOutput(X_mb), T_mb))

In [21]:
# Plot the loss over the iterations
fig = plt.figure(figsize=(5, 3))
plt.plot(ls_of_loss, 'b-')
plt.xlabel('minibatch iteration')
plt.ylabel('$\\xi$', fontsize=15)
plt.title('Decrease of loss over backprop iteration')
plt.xlim(0, 100)
fig.subplots_adjust(bottom=0.2)
plt.show()

<IPython.core.display.Javascript object>

### Test

In [22]:
nb_test = 5
Xtest, Ttest = create_dataset(nb_test, sequence_len)

Y = RNN.getBinaryOutput(Xtest)
Yf = RNN.getOutput(Xtest)

for i in range(Xtest.shape[0]):
    printSample(Xtest[i,:,0], Xtest[i,:,1], Ttest[i,:,:], Y[i,:,:])
    print('')

x1:   0100010   34
x2: + 1100100   19
      -------   --
t:  = 1010110   53
y:  = 1010110

x1:   1010100   21
x2: + 1110100   23
      -------   --
t:  = 0011010   44
y:  = 0011010

x1:   1111010   47
x2: + 0000000    0
      -------   --
t:  = 1111010   47
y:  = 1111010

x1:   1000000    1
x2: + 1111110   63
      -------   --
t:  = 0000001   64
y:  = 0000001

x1:   1010100   21
x2: + 1010100   21
      -------   --
t:  = 0101010   42
y:  = 0101010



#***Second Section

##Importing Libraries

In [23]:
pip install nltk

In [24]:
pip install markovify

In [25]:
import re
import nltk
import random
import markovify
import numpy as np
import tensorflow as tf
import keras.utils as ku
from nltk import corpus
from keras.utils import np_utils
from nltk.corpus import stopwords
from keras.layers import Embedding
from nltk.tokenize import word_tokenize
from keras.layers import Dropout, Flatten
from keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Activation, Dense, LSTM
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##1)Markov chain text generation

In [26]:
empty_list = []
stop_words = 'و در به از كه مي اين است را با هاي براي آن يك شود شده خود ها كرد شد اي تا كند بر بود گفت نيز وي هم كنند دارد ما كرده يا اما بايد دو اند هر خواهد او مورد آنها باشد ديگر مردم نمي بين پيش پس اگر همه صورت يكي هستند بي من دهد هزار نيست استفاده داد داشته راه داشت چه همچنين كردند داده بوده دارند همين ميليون سوي شوند بيشتر بسيار روي گرفته هايي تواند اول نام هيچ چند جديد بيش شدن كردن كنيم نشان حتي اينكه ولی توسط چنين برخي نه ديروز دوم درباره بعد مختلف گيرد شما گفته آنان بار طور گرفت دهند گذاري بسياري طي بودند ميليارد بدون تمام كل تر براساس شدند ترين امروز باشند ندارد چون قابل گويد ديگري همان خواهند قبل آمده اكنون تحت طريق گيري جاي هنوز چرا البته كنيد سازي سوم كنم بلكه زير توانند ضمن فقط بودن حق آيد وقتي اش يابد نخستين مقابل خدمات امسال تاكنون مانند تازه آورد فكر آنچه نخست نشده شايد چهار جريان پنج ساخته زيرا نزديك برداري كسي ريزي رفت گردد مثل آمد ام بهترين دانست كمتر دادن تمامي جلوگيري بيشتري ايم ناشي چيزي آنكه بالا بنابراين ايشان بعضي دادند داشتند برخوردار نخواهد هنگام نبايد غير نبود ديده وگو داريم چگونه بندي خواست فوق ده نوعي هستيم ديگران همچنان سراسر ندارند گروهي سعي روزهاي آنجا يكديگر كردم بيست بروز سپس رفته آورده نمايد باشيم گويند زياد خويش همواره گذاشته شش نداشته شناسي خواهيم آباد داشتن نظير همچون باره نكرده شان سابق هفت دانند جايي بی جز زیرِ رویِ سریِ تویِ جلویِ پیشِ عقبِ بالایِ خارجِ وسطِ بیرونِ سویِ کنارِ پاعینِ نزدِ نزدیکِ دنبالِ حدودِ برابرِ طبقِ مانندِ ضدِّ هنگامِ برایِ مثلِ بارة اثرِ تولِ علّتِ سمتِ عنوانِ قصدِ روب جدا کی که چیست هست کجا کجاست کَی چطور کدام آیا مگر چندین یک چیزی دیگر کسی بعری هیچ چیز جا کس هرگز یا تنها بلکه خیاه بله بلی آره آری مرسی البتّه لطفاً ّه انکه وقتیکه همین پیش مدّتی هنگامی مان تان ۰ ۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹ … ! '' # ( ) * , - . / : [ ] « » ، ؛ ؟ '
with open('/content/poetry.txt', 'r') as fh:
    for line in fh:
        text_tokens = word_tokenize(line)
        tokens_without_sw = [word for word in text_tokens if not word in stop_words]
        filtered_sentence = (" ").join(tokens_without_sw)
        filtered_paragraph = ('\n').join(filtered_sentence)
        empty_list.append(filtered_sentence)
    #print(empty_list)

text = '\n'.join(empty_list)

In [27]:
model = markovify.NewlineText(text)

In [28]:
for i in range(5):
    print()
    for i in range(random.randrange(1, 4)):
        print(model.make_short_sentence(30))


برست ولوله باغ بوستان نیست

None
دیده دیدار دوست جهانش بهاست
بیند دست آغوش مرا

None
جور کنی جور نیست بنالی جفاست
ستمگرا دل سعدی فرونشوید عشق

جهل جوانان پنجه کردن جوان

کعبه دیدار دوست جهانش بهاست


###Data Preprocessing

####Delete StopWords

In [29]:
empty_list = []
stop_words = 'و در به از كه مي اين است را با هاي براي آن يك شود شده خود ها كرد شد اي تا كند بر بود گفت نيز وي هم كنند دارد ما كرده يا اما بايد دو اند هر خواهد او مورد آنها باشد ديگر مردم نمي بين پيش پس اگر همه صورت يكي هستند بي من دهد هزار نيست استفاده داد داشته راه داشت چه همچنين كردند داده بوده دارند همين ميليون سوي شوند بيشتر بسيار روي گرفته هايي تواند اول نام هيچ چند جديد بيش شدن كردن كنيم نشان حتي اينكه ولی توسط چنين برخي نه ديروز دوم درباره بعد مختلف گيرد شما گفته آنان بار طور گرفت دهند گذاري بسياري طي بودند ميليارد بدون تمام كل تر براساس شدند ترين امروز باشند ندارد چون قابل گويد ديگري همان خواهند قبل آمده اكنون تحت طريق گيري جاي هنوز چرا البته كنيد سازي سوم كنم بلكه زير توانند ضمن فقط بودن حق آيد وقتي اش يابد نخستين مقابل خدمات امسال تاكنون مانند تازه آورد فكر آنچه نخست نشده شايد چهار جريان پنج ساخته زيرا نزديك برداري كسي ريزي رفت گردد مثل آمد ام بهترين دانست كمتر دادن تمامي جلوگيري بيشتري ايم ناشي چيزي آنكه بالا بنابراين ايشان بعضي دادند داشتند برخوردار نخواهد هنگام نبايد غير نبود ديده وگو داريم چگونه بندي خواست فوق ده نوعي هستيم ديگران همچنان سراسر ندارند گروهي سعي روزهاي آنجا يكديگر كردم بيست بروز سپس رفته آورده نمايد باشيم گويند زياد خويش همواره گذاشته شش نداشته شناسي خواهيم آباد داشتن نظير همچون باره نكرده شان سابق هفت دانند جايي بی جز زیرِ رویِ سریِ تویِ جلویِ پیشِ عقبِ بالایِ خارجِ وسطِ بیرونِ سویِ کنارِ پاعینِ نزدِ نزدیکِ دنبالِ حدودِ برابرِ طبقِ مانندِ ضدِّ هنگامِ برایِ مثلِ بارة اثرِ تولِ علّتِ سمتِ عنوانِ قصدِ روب جدا کی که چیست هست کجا کجاست کَی چطور کدام آیا مگر چندین یک چیزی دیگر کسی بعری هیچ چیز جا کس هرگز یا تنها بلکه خیاه بله بلی آره آری مرسی البتّه لطفاً ّه انکه وقتیکه همین پیش مدّتی هنگامی مان تان ۰ ۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹ … ! '' # ( ) * , - . / : [ ] « » ، ؛ ؟ '
with open('/content/poetry.txt', 'r') as fh:
    for line in fh:
        text_tokens = word_tokenize(line)
        tokens_without_sw = [word for word in text_tokens if not word in stop_words]
        filtered_sentence = (" ").join(tokens_without_sw)
        filtered_paragraph = ('\n').join(filtered_sentence)
        empty_list.append(filtered_sentence)
    #print(empty_list)

text = '\n'.join(empty_list)

####Delete Extra Space

In [30]:
re.sub(' +', ' ', text)

'\n\nدفتر ایزد دانا\nصانع پروردگار حی توانا\nاکبر اعظم خدای عالم آدم\nخوب آفرید سیرت زیبا\nبخشندگی بنده نوازی\nمرغ هوا نصیب ماهی دریا\nقسمت می\u200cخورند منعم درویش\nروزی می\u200cبرند پشه عنقا\nحاجت موری علم غیب بداند\nچاهی صخره صما\nجانور نطفه می\u200cکند شکر نی\nبرگ\u200cتر چوب خشک چشمه خارا\nشربت نوش آفرید مگس نحل\nنخل تناور کند دانه خرما\nهمگان بی\u200cنیاز مشفق\nعالم نهان پیدا\nپرتو نور سرادقات جلالش\nعظمت ماورای فکرت دانا\nزبان دهان عارف مدهوش\nحمد ثنا می\u200cکند موی اعضا\nنداند سپاس نعمت\nحیف نصیب رحمت فردا\nبارخدایا مهیمنی مدبر\nعیبی مقدسی مبرا\nنتوانیم حمد گفتن\nکروبیان عالم\nسعدی فهم اوست سخن\nکمال وهم رسد\n\nنفس خرم صبا\nیار آمده\u200cای مرحبا\nقافله شب شنیدی صبح\nمرغ سلیمان خبر سبا\nخشمست حریف\nسخنی می\u200cرود اندر رضا\nصلح آمده\u200cای خلاف\nقدم خوف روم رجا\nدگر کوی دوست\nبگذری پیک نسیم صبا\nرمقی بیش نماند ضعیف\nکند بی\u200cجان بقا\nدلداری پیمان عهد\nنیک نکردی نکردی وفا\nلیکن دور وصالی\nصلح فراموش کند ماجرا\nگریبان نرسد دست مرگ\nدست دامن نکنیمت رها\nدوست نباشد حقیقت\nدوس

####Convert word to index

In [31]:
chars = list(set(text))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

vocab_size = len(chars)
print('Vocabulary size: {}'.format(vocab_size))

Vocabulary size: 42


####Data preparation


In [32]:
X = [] # training array
Y = [] # target array

length = len(text)
seq_length = 100 # number of characters to consider before predicting a character

# Iterate over length of text and create sequences stored in X, true values stored in Y
# true values being which character would actually come after sequence stored in X
for i in range(0, length - seq_length, 1):
    sequence = text[i:i + seq_length]
    label = text[i + seq_length]
    X.append([char_indices[char] for char in sequence])
    Y.append(char_indices[label])

print('Number of sequences: {}'.format(len(X)))

Number of sequences: 42597


####Reshape dimensions

In [33]:
X_new = np.reshape(X, (len(X), seq_length, 1))
# Scale values
X_new = X_new/float(len(chars))
# One-hot encode Y to remove ordinal relationships
Y_new = np_utils.to_categorical(Y)

X_new.shape, Y_new.shape

((42597, 100, 1), (42597, 42))

##2)Character-Level LSTM Text Generation

###Building RNN

In [34]:
model = Sequential()

model.add(LSTM(150, input_shape = (X_new.shape[1], X_new.shape[2]), return_sequences = True))

model.add(Dropout(0.1))

model.add(Flatten())

model.add(Dense(Y_new.shape[1], activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit(X_new, Y_new, epochs = 1, verbose = 1)

1332/1332 [==============================] - 146s 108ms/step - loss: 3.0945


###Generate text

In [35]:
# Random start
start = np.random.randint(0, len(X)-1)
string_mapped = list(X[start])
full_string = [indices_char[value] for value in string_mapped]

# Generate text
for i in range(400):
    x = np.reshape(string_mapped, (1, len(string_mapped), 1))
    x = x / float(len(chars))
    
    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [indices_char[value] for value in string_mapped]
    full_string.append(indices_char[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
# Combine text
newtext = ''
for char in full_string:
    newtext = newtext + char

print(newtext)

رک بسپارند
چشم ترک بینند زلف هندو
مغان خدمت می‌کنند فرخار
ندیده‌اند دلبران
حصار قلعه باغی منجنیق
باماا ن ا د اا با ااا ت
 ت ا اا  اا  ا ا ت
 ن
 ن د  ن   ن  ایااا
 ن ا ن  ا   ا  اا ا ا ا ن ن   ا   ا   ا  ا ا ا ا   ا   ا   ا  ا  ا ا  ا   ا       ا ا  ا   ا       ا   ا    ا   ا           ا   ا   ا                   ا                                                                                                                                                                                         


####Predicting Next Words

In [36]:
# Lowercase all text
# text = corpus.lower()
text = text.split('\n')

# Create Tokenizer object to convert words to sequences of integers
tokenizer = Tokenizer(num_words = None, filters = '#$%&()*+-<=>@[\\]^_`{|}~\t\n', lower = False)

# Train tokenizer to the texts
tokenizer.fit_on_texts(text)
total_words = len(tokenizer.word_index) + 1

# Convert list of strings into flat dataset of sequences of tokens
sequences = []
for line in text:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Pad sequences to ensure equal lengths
max_seq_len = max([len(x) for x in sequences])
sequences = np.array(pad_sequences(sequences, maxlen = max_seq_len, padding = 'pre'))

# Create n-grams sequence predictors and labels
predictors, label = sequences[:, :-1], sequences[:, -1]
label = ku.to_categorical(label, num_classes = total_words)

####Creat Model

In [37]:
# Input layer takes sequence of words as input
input_len = max_seq_len - 1
model = Sequential()
model.add(Embedding(total_words, 10, input_length = input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Use 10 epoch for efficacy
model.fit(predictors, label, epochs = 10, verbose = 1)

Epoch 1/10
188/188 [==============================] - 5s 17ms/step - loss: 7.9572
Epoch 2/10
188/188 [==============================] - 3s 17ms/step - loss: 7.4828
Epoch 3/10
188/188 [==============================] - 3s 17ms/step - loss: 7.3567
Epoch 4/10
188/188 [==============================] - 3s 17ms/step - loss: 7.2959
Epoch 5/10
188/188 [==============================] - 3s 17ms/step - loss: 7.2420
Epoch 6/10
188/188 [==============================] - 3s 17ms/step - loss: 7.1854
Epoch 7/10
188/188 [==============================] - 3s 16ms/step - loss: 7.0518
Epoch 8/10
188/188 [==============================] - 3s 17ms/step - loss: 6.9122
Epoch 9/10
188/188 [==============================] - 3s 17ms/step - loss: 6.7786
Epoch 10/10
188/188 [==============================] - 3s 17ms/step - loss: 6.6266


###Generate Line

In [38]:
# Function to generate line
def generate_line(text, next_words, max_seq_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen = max_seq_len - 1, padding = 'pre')
        predicted = model.predict_classes(token_list, verbose = 0)
        
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text += ' ' + output_word
    return text

####Line generating output

In [39]:
generate_line("مرغ هوا را", 5, max_seq_len, model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'مرغ هوا را این نخواهم مرا دوست دوست'

In [40]:
generate_line("نخل تناور کند", 5, max_seq_len, model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'نخل تناور کند دوست مرا آمده\u200cست دوست دوست'

In [41]:
generate_line("حمد و ثنا", 5, max_seq_len, model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'حمد و ثنا این برافکن آمده\u200cست اوست اوست'

In [42]:
generate_line("ما نتوانیم حق", 5, max_seq_len, model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'ما نتوانیم حق دوست سعدی دوست دوست دوست'

In [43]:
generate_line("سلام من سحر", 5, max_seq_len, model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'سلام من سحر دوست سعدی این خوش آمده\u200cست'

#***Extra Work

In [44]:
list = []
stop_words = 'و در به از كه مي اين است را با هاي براي آن يك شود شده خود ها كرد شد اي تا كند بر بود گفت نيز وي هم كنند دارد ما كرده يا اما بايد دو اند هر خواهد او مورد آنها باشد ديگر مردم نمي بين پيش پس اگر همه صورت يكي هستند بي من دهد هزار نيست استفاده داد داشته راه داشت چه همچنين كردند داده بوده دارند همين ميليون سوي شوند بيشتر بسيار روي گرفته هايي تواند اول نام هيچ چند جديد بيش شدن كردن كنيم نشان حتي اينكه ولی توسط چنين برخي نه ديروز دوم درباره بعد مختلف گيرد شما گفته آنان بار طور گرفت دهند گذاري بسياري طي بودند ميليارد بدون تمام كل تر براساس شدند ترين امروز باشند ندارد چون قابل گويد ديگري همان خواهند قبل آمده اكنون تحت طريق گيري جاي هنوز چرا البته كنيد سازي سوم كنم بلكه زير توانند ضمن فقط بودن حق آيد وقتي اش يابد نخستين مقابل خدمات امسال تاكنون مانند تازه آورد فكر آنچه نخست نشده شايد چهار جريان پنج ساخته زيرا نزديك برداري كسي ريزي رفت گردد مثل آمد ام بهترين دانست كمتر دادن تمامي جلوگيري بيشتري ايم ناشي چيزي آنكه بالا بنابراين ايشان بعضي دادند داشتند برخوردار نخواهد هنگام نبايد غير نبود ديده وگو داريم چگونه بندي خواست فوق ده نوعي هستيم ديگران همچنان سراسر ندارند گروهي سعي روزهاي آنجا يكديگر كردم بيست بروز سپس رفته آورده نمايد باشيم گويند زياد خويش همواره گذاشته شش نداشته شناسي خواهيم آباد داشتن نظير همچون باره نكرده شان سابق هفت دانند جايي بی جز زیرِ رویِ سریِ تویِ جلویِ پیشِ عقبِ بالایِ خارجِ وسطِ بیرونِ سویِ کنارِ پاعینِ نزدِ نزدیکِ دنبالِ حدودِ برابرِ طبقِ مانندِ ضدِّ هنگامِ برایِ مثلِ بارة اثرِ تولِ علّتِ سمتِ عنوانِ قصدِ روب جدا کی که چیست هست کجا کجاست کَی چطور کدام آیا مگر چندین یک چیزی دیگر کسی بعری هیچ چیز جا کس هرگز یا تنها بلکه خیاه بله بلی آره آری مرسی البتّه لطفاً ّه انکه وقتیکه همین پیش مدّتی هنگامی مان تان ۰ ۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹ … ! '' # ( ) * , - . / : [ ] « » ، ؛ ؟ '
with open('/content/poetry.txt', 'r') as fh:
    for line in fh:
        text_tokens = word_tokenize(line)
        tokens_without_sw = [word for word in text_tokens if not word in stop_words]
        filtered_sentence = (" ").join(tokens_without_sw)
        filtered_paragraph = ('\n').join(filtered_sentence)
        list.append(filtered_sentence)
    #print(list)

text = '\n'.join(list)

characters = sorted(set(text))   #filter out all the unique character

char_to_index = dict((c, i) for i, c in enumerate(characters))  #assigne one number to each character -> number as a key
index_to_char = dict((i, c) for i, c in enumerate(characters))  #character as a value

SEQ_LENGTH = 40 
STEP_SIZE = 3  

sentences = []
next_char = []

#fill up the link based on the seq_length and step_size
#it start from first until the last seq and always have the step_size in between
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    #add the sentence -> i:starting position/ till the sequence we wanna add to sentence -> 0-4
    sentences.append(text[i: i + SEQ_LENGTH])
    #specify the next character -> 5
    next_char.append(text[i + SEQ_LENGTH])

#change the character and string format to numerical format
#one dimension for all of the possible sentences that we have
#one dimension for all the individual positions in the sentences and 
#one more positions for all the possible characters that we can have

x = np.zeros((len(sentences), #how many sentences they are
              SEQ_LENGTH, #the length of the sentences
              len(characters)), #the amount of possible characters
             dtype=np.bool)

y = np.zeros((len(sentences),
              len(characters)), 
             dtype=np.bool)


#fill up this erase 
for i, satz in enumerate(sentences):  #taking all the sequences we have just created
    for t, char in enumerate(satz):   #for each of those sentenceses we enumerate all the character in those sentences
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.01))

model.fit(x, y, batch_size=256, epochs=4) #how many examples we put in to the network at once


#takes the prediction of our model and fix one character
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def generate_text(length, temperature):
    #starting text provide the input for our NN to predict the first next character
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        #prediction
        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated


print("----------0.2--------")
print(generate_text(300, 0.2))
print("----------0.4--------")
print(generate_text(300, 0.4))
print("----------0.5--------")
print(generate_text(300, 0.5))
print("----------0.6--------")
print(generate_text(300, 0.6))
print("----------0.7--------")
print(generate_text(300, 0.7))
print("----------0.8--------")
print(generate_text(300, 0.8))

Epoch 1/4
56/56 [==============================] - 11s 158ms/step - loss: 3.2074
Epoch 2/4
56/56 [==============================] - 9s 158ms/step - loss: 2.6500
Epoch 3/4
56/56 [==============================] - 9s 158ms/step - loss: 2.4637
Epoch 4/4
56/56 [==============================] - 9s 158ms/step - loss: 2.3390
----------0.2--------
سن طبل عشق فروکوفت
بانگ برآمد غارت دل دیدی می‌کند می‌کند می‌رود می‌کند می‌باند بی‌کند می‌بوی می‌کند می‌کند می‌کند می‌برد می‌رود می‌کند می‌کند می‌کند می‌بوش می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌بود می‌باند می‌کند می‌کند می‌برد می‌باری می‌رود بی‌اند می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌کند می‌رود می‌رود 
----------0.4--------
بیند خواب
ببستی چشم سحارت
بدین چشم خواب برست بازیان بی‌سان می‌رود می‌کند بی‌روان جان
نمی‌ورد می‌نود دان عشق آرخان بی‌اوی دل بوست
گوی دین می‌توان برخوان می‌کند می‌کند می‌بانی می‌نهان برآت می‌رود دست
برو سعدی نیای می‌ندی بی‌کند ملال

برخت دل بوز خوش می‌کند بی‌رود وشم نیمین نمی‌کند می‌رود می‌ندی 